# Model.ipynb
This file contains all the necessary features to build our model from scratch:
- Generating and Saving Data
- Loading the Data
- Building the Model Structure
- Compiling the Model
- Fitting/Training the Model
- Saving the Model


## Generate Data

In [ ]:
import chess
import chess.engine
import random
import numpy as np
import os

# This creates randoms boards we will feed to our model
def gen_random_board(max_moves=200):
    board = chess.Board()
    depth = random.randrange(0, max_moves)

    for _ in range(depth):
        all_possible_moves = list(board.legal_moves)
        move = random.choice(all_possible_moves)
        board.push(move)
        if board.is_game_over():
            break
    
    return board

# This returns the Stockfish eval which we will feed to our model
def stockfish(board, depth):
    with chess.engine.SimpleEngine.popen_uci("/usr/games/stockfish") as sf:
        result = sf.analyse(board, chess.engine.Limit(depth=depth))
        score = result['score'].white().score()
        return score

In [ ]:
board = gen_random_board()
board

In [ ]:
print(stockfish(board, 10))

In [ ]:
squares_index = {
    'a': 0,
    'b': 1,
    'c': 2,
    'd': 3,
    'e': 4,
    'f': 5,
    'g': 6,
    'h': 7,
}

def square_to_index(square):
    letter = chess.square_name(square)
    return 8 - int(letter[1]), squares_index[letter[0]]

def convert_to_3d(board):
    board3d = np.zeros((14,8,8), dtype=np.int8)

    for piece in chess.PIECE_TYPES:
        for square in board.pieces(piece, chess.WHITE):
            idx = np.unravel_index(square, (8,8))
            board3d[piece - 1][7 - idx[0]][idx[1]] = 1
        for square in board.pieces(piece, chess.BLACK):
            idx = np.unravel_index(square, (8,8))
            board3d[piece + 5][7 - idx[0]][idx[1]] = 1

    aux = board.turn
    board.turn = chess.WHITE
    for move in board.legal_moves:
        i, j, = square_to_index(move.to_square)
        board3d[12][i][j] = 1
    board.turn = chess.BLACK
    for move in board.legal_moves:
        i, j = square_to_index(move.to_square)
        board3d[13][i][j] = 1
    board.turn = aux
    
    return board3d

In [ ]:
boards = np.arange(89600000, dtype=np.int8).reshape(100000, 14, 8, 8)
evals = np.arange(100000, dtype=np.int32)

for i in range(100000):
    board = gen_random_board()
    eval = stockfish(board, 10)

    eval = eval if eval else 0
    
    boards[i] = convert_to_3d(board)
    evals[i] = eval

file_name = 'data.npz'
np.savez(os.path.join('data', file_name), b=boards, v=evals)

## Load Data

In [1]:
import numpy as np
import os

file_name = 'data.npz'  # Change this to use a different dataset from the data folder

def get_dataset():
    data = np.load(os.path.join('data', file_name))
    b, v = data['b'], data['v']
    v = np.asarray( v / abs(v).max() / 2 + 0.5, dtype=np.float32)

    return b, v

x_train, y_train = get_dataset()

def get_test_dataset():
    data = np.load('data/testdata.npz')
    b, v = data['b'], data['v']
    v = np.asarray( v / abs(v).max() / 2 + 0.5, dtype=np.float32)

    return b, v

x_test, y_test = get_test_dataset()


## AI Convolution Neural Network Model

### Build Model

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
import tensorflow.keras.layers as layers

inputs = layers.Input(shape=(14, 8, 8))

x = layers.Conv2D(filters=32, kernel_size=3, padding='same', activation=tf.nn.sigmoid)(inputs)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(filters=32, kernel_size=3, padding='same', activation=tf.nn.sigmoid)(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(filters=64, kernel_size=3, padding='same', activation=tf.nn.sigmoid)(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(filters=64, kernel_size=3, padding='same', activation=tf.nn.sigmoid)(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(filters=128, kernel_size=3, padding='same', activation=tf.nn.sigmoid)(x)

x = layers.Flatten()(x)
x = layers.Dense(32, tf.nn.sigmoid)(x)

outputs = layers.Dense(1, tf.nn.sigmoid)(x)

model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
model.summary()

2023-12-04 21:12:21.525468: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-04 21:12:21.525518: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-04 21:12:21.525557: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 14, 8, 8)]        0         
                                                                 
 conv2d (Conv2D)             (None, 14, 8, 32)         2336      
                                                                 
 batch_normalization (Batch  (None, 14, 8, 32)         128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 8, 32)         9248      
                                                                 
 batch_normalization_1 (Bat  (None, 14, 8, 32)         128       
 chNormalization)                                                
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 8, 64)         18496 

### Compile Model

In [3]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss=tf.keras.losses.MeanSquaredError())

### Fit/Train Model

In [4]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import datetime

import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

log_name = 'Chess-CNN-Model' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = os.path.join('logs', log_name)
tb = TensorBoard(log_dir=log_dir) 

checkpoint_filepath = '/tmp/checkpoint/'

callbacks = [
    tb,
    
    ModelCheckpoint(
        filepath = checkpoint_filepath,
        save_weights_only = False,
        save_best_only = True,
    )
]

model.fit(x_train, y_train, batch_size=32, epochs=10, verbose=1, validation_split=0.1, callbacks=callbacks, validation_data=(x_test, y_test), validation_batch_size=32)


ValueError: Data cardinality is ambiguous:
  x sizes: 100000
  y sizes: 89600000
Make sure all arrays contain the same number of samples.

### Save Model
- If we like the performance of our model, we should save it
- The following line of code saves our model to a '.keras' file
    - We can load this back in and train it more
    - Or we can load it into our chess game and play against it

In [ ]:
model_file = 'model' +  str(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")) + '.keras'
model.save(os.path.join('model', model_file))